## Narration

Narration is yet small part of functionality that is going to be a huge extension point of the project. `INarrator` should:

* Host the character's state as a dictionary of variables
* Change this state by external commands from the console
* Deduce the voice of the character for the `DubbingService` and tags to filter images in `ImageService` from the state
* (in the future) autonomously change the state based on environment (time of day, day of the week, weather), thus making the character to live its own life inside Avatar service.

The most obvious and already implemented test case for this state is the character itself: if we have multiple characters, the current character will be defined as a variable in the state, thus determining the voice and the appropriate images. `SimpleNarrator` does exactly this.


In [1]:
from kaia.avatar.narrator import SimpleNarrator

narrator = SimpleNarrator('Lina', {'Lina': 'p225', 'Jug': 'p229'})

Now we can configure and test the whole Avatar web server. Note that Avatar server goes together with BrainBox, so we will have to write a scenario for the both services together.

In [3]:
from kaia.brainbox import BrainBox
from kaia.avatar import BrainBoxDubbingService, AvatarSettings,  AvatarTestApi, AvatarSettings, ImageService, open_tts_task_generator, AnyImageStrategy
from pathlib import Path
from kaia.infra import Loc
from ipywidgets import Audio, Image, VBox

with BrainBox().create_test_api() as bb_api:
    dubbing_service = BrainBoxDubbingService(open_tts_task_generator, bb_api)
    with Loc.create_temp_file('demos','json') as feedback_file:
        image_service = ImageService(AnyImageStrategy(), Path('files/image_library.zip'), feedback_file)
        settings = AvatarSettings()
        with AvatarTestApi(settings, narrator, dubbing_service, image_service) as api:
            image_lina = api.image_get()
            audio_lina = api.dub_string('Hello, my name is Lina')
            api.state_change(dict(character='Jug'))
            image_jug = api.image_get()
            audio_jug = api.dub_string('Hello, my name is Jug')



In [4]:
VBox([
    Image(value=image_lina.data, width=100, height=100),
    Audio(value=audio_lina.data, autoplay=False),
    Image(value=image_jug.data, width=100, height=100),
    Audio(value=audio_jug.data, autoplay=False),
])